### **Rede Neural Artificial**

Importando as bibliotecas

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
tf.__version__

'2.9.1'

##### **Parte 1 - Pré-processamento dos dados** 

**Importando o banco de dados**

In [5]:
df = pd.read_csv('files\Churn_Modelling.csv', sep=',')

In [7]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [8]:
X = df.iloc[:, 3:-1].values # matriz de recursos - 'CreditScore' a 'EstimatedSalary'
y = df.iloc[:,-1].values # vetor variável dependente - 'Exited'

In [9]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [10]:
print(y)

[1 0 1 ... 1 1 0]


**Codificação dos dados categóricos**

Coluna 'Gender'

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [12]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


Coluna 'Geography'

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [15]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


**Dividindo o conjunto de dados em Treinamento e Teste**

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**Dimensionamento dos daados**

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

##### **Parte 2 - Contruíndo uma rede neural artificial - RNA** 

**Inicializando a RNA**

In [18]:
rna = tf.keras.models.Sequential()

**Adicionando a camada de entrada e a primeira camada oculta**

In [19]:
rna.add(tf.keras.layers.Dense(units=6, activation='relu')) # função de ativação retificadora

**Adicionando a segunda camada oculta**


In [20]:
rna.add(tf.keras.layers.Dense(units=6, activation='relu')) # função de ativação retificadora

**Adicionando a camada oculta de saída**


In [21]:
rna.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # função de ativação sigmodal

- activation'sigmoid'= para classificação binária

- activation 'softmax' = para classificação não binária.

##### **Parte 3 - Treinando a RNA** 

**Compilando a RNA**

In [22]:
rna.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # parâmetros para avaliar a RNA

- Optimizer = otimiza o 'Gradient Descent' estocastico. Atualiza os pesos para reduzir o erro de perdas entre previsões e resultados reais.

- loss function = calcula a diferença entre as previsões e o resultado real.

- 'binary_crossentropy' = função de perda ideal para utilizar em casos de classificação binária.

- 'category_crossentropy' = função de perda adequada para classificação não binária.

- metrics = precisão, métrica de avaliação final.

**Treinando a RNA**

In [23]:
rna.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 3s 2ms/step - loss: 0.5385 - accuracy: 0.7934
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4690 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4523 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4419 - accuracy: 0.7959
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4344 - accuracy: 0.8025
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4285 - accuracy: 0.8087
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4220 - accuracy: 0.8177
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4153 - accuracy: 0.8234
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4095 - accuracy: 0.8263
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4054 - accura

##### **Parte 4 - Produzindo predições e avaliando o modelo** 


In [24]:
print(rna.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

1/1 [==============================] - 0s 305ms/step
[[0.02652202]]


A probabilidade prevista para que esse cliente saia do banco é de 2,6%, ou seja, baixa chance desse cliente deixar o banco. 

In [25]:
print(rna.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 41ms/step
[[False]]


- Se > 0.5 = o cliente fica no banco.
- Se < 0.5 = o cliente deixa o banco.

**Predições de resultados com os dados de Teste**

In [26]:
y_pred = rna.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 911us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


**Precisão final**

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1522   73]
 [ 207  198]]


0.86

- Precisão de 86%.